In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv('Social_Network_Ads.csv')

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset = pd.get_dummies(dataset,drop_first = True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [6]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [7]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [8]:
dataset['Purchased'].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [9]:
indep = dataset[['User ID', 'Age', 'EstimatedSalary', 'Gender_Male']]
dep = dataset['Purchased']

In [10]:
indep.shape

(400, 4)

In [11]:
indep

,User ID,Age,EstimatedSalary,Gender_Male
0,15624510,19,19000,True
1,15810944,35,20000,True
2,15668575,26,43000,False
3,15603246,27,57000,False
4,15804002,19,76000,True
...,...,...,...,...
395,15691863,46,41000,False
396,15706071,51,23000,True
397,15654296,50,20000,False
398,15755018,36,33000,True


In [12]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(indep,dep, test_size = 0.30, random_state = 0)

In [14]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [15]:
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV

param_grid = {'n_neighbors':[1,5,7,10],
              'metric':['minkowski'],
              'weights':['uniform', 'distance'],
              'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'],
              'p':[2]
             } 



grid = GridSearchCV(KNeighborsClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 

grid.fit(X_train, Y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'metric': ['minkowski'], 'n_neighbors': [1, 5, 7, 10],
                         'p': [2], 'weights': ['uniform', 'distance']},
             scoring='f1_weighted', verbose=3)

In [34]:
result=grid.cv_results_
#print(result)

Y_pred = grid.predict(X_test)
#print(y_pred)
   
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(Y_test, Y_pred)
#print(cm)

from sklearn.metrics import classification_report

clf_report = classification_report(Y_test, Y_pred)

#print(clf_report)

In [36]:
from sklearn.metrics import f1_score

f1_macro=f1_score(Y_test,Y_pred,average='weighted')

print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'algorithm': 'auto', 'metric': 'minkowski', 'n_neighbors': 7, 'p': 2, 'weights': 'distance'}: 0.9161458333333333


In [38]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[75  4]
 [ 6 35]]


In [40]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.93      0.95      0.94        79
           1       0.90      0.85      0.88        41

    accuracy                           0.92       120
   macro avg       0.91      0.90      0.91       120
weighted avg       0.92      0.92      0.92       120



In [42]:
#AUC ROC stands for “Area Under the Curve” of the “Receiver Operating Characteristic” curve. 
from sklearn.metrics import roc_auc_score

roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])

0.9546156221055881

In [44]:
table=pd.DataFrame.from_dict(result)

table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_metric,param_n_neighbors,param_p,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.272705,0.135586,0.076654,0.033274,auto,minkowski,1,2,uniform,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.891485,0.799537,0.842217,0.821429,0.872761,0.845486,0.033349,21
1,0.001959,0.003917,0.021779,0.008605,auto,minkowski,1,2,distance,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.891485,0.799537,0.842217,0.821429,0.872761,0.845486,0.033349,21
2,0.004525,0.002127,0.027610,0.006440,auto,minkowski,5,2,uniform,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.872432,0.855314,0.858503,0.910254,0.888158,0.876932,0.020315,5
3,0.004863,0.002767,0.010350,0.003642,auto,minkowski,5,2,distance,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.872432,0.855314,0.841398,0.892857,0.888158,0.870032,0.019452,9
4,0.002103,0.004082,0.030742,0.011027,auto,minkowski,7,2,uniform,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.872432,0.874254,0.835985,0.891667,0.870721,0.869012,0.018145,13
5,0.004059,0.004972,0.016151,0.004961,auto,minkowski,7,2,distance,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.872432,0.874254,0.875644,0.910254,0.870721,0.880661,0.014890,1
6,0.008077,0.004039,0.024509,0.004897,auto,minkowski,10,2,uniform,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.829361,0.816207,0.692857,0.890114,0.868182,0.819344,0.068560,29
7,0.004080,0.004998,0.012390,0.004379,auto,minkowski,10,2,distance,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.852966,0.874254,0.821429,0.891667,0.888158,0.865694,0.025977,17
8,0.024554,0.018211,0.040171,0.010825,ball_tree,minkowski,1,2,uniform,"{'algorithm': 'ball_tree', 'metric': 'minkowsk...",0.891485,0.799537,0.842217,0.821429,0.872761,0.845486,0.033349,21
9,0.005970,0.005621,0.016848,0.003218,ball_tree,minkowski,1,2,distance,"{'algorithm': 'ball_tree', 'metric': 'minkowsk...",0.891485,0.799537,0.842217,0.821429,0.872761,0.845486,0.033349,21
